<img src="https://i.postimg.cc/ZRSDQWPG/pokeapi-pyspark.png" width = "500"/> <center>

# Consumindo e analisando dados da PokéApi utilizando Python e Pyspark

## Importação de bibliotecas

In [0]:
import requests
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, coalesce, count, explode_outer, when, row_number, sum as _sum, min as _min, max as _max, round as _round
from pyspark.sql.window import Window

#Create PySpark SparkSession
spark = SparkSession.builder.appName("Spark App").getOrCreate()

## Extração dos dados da API

Para fazer a extração dos dados, o primeiro passo foi interpretar as informações contidas na [Documentação](https://pokeapi.co/docs/v2) da PokéAPI, os recursos nela disponíveis e seus atributos, ao passo que foram levantadas as informações necessárias para responder às perguntas propostas pelo problema.
Notou-se que todos os dados necessários estavam contidos em alguns Endpoints distribuídos nos seguintes grupos:
- [Evolution](https://pokeapi.co/docs/v2#evolution-section)
- [Pokémon](https://pokeapi.co/docs/v2#pokemon-section)
- [Games](https://pokeapi.co/docs/v2#games-section)

Foi, então, criada uma função para automatizar o processo de fazer as requisições para cada recurso disponível nos Endpoints desejados até que todos os recursos fossem extraídos. Para isso, a função obtém primeiramente o número de recursos disponibilizados pela API em cada Endpoint e utiliza esse valor para obter as URLs de cada recurso e fazer as requisições de dados para cada um deles. Em seguida, é gerado um Spark Dataframe com os dados extraídos.

In [0]:
# Criando função geral de extração de dados e geração de dataframes
def generate_dataframe_from_request(endpoint_name):
    """
    Função que faz a requisição na PokéAPI de todos os
    recursos de um endpoint recebido como parâmetro
    (endpoit_name) e gera um spark dataframe com os dados.
    """
    
    # Faz a primeira requisição para obter o número de recursos disponíveis
    max_page = (requests.get(f'https://pokeapi.co/api/v2/{endpoint_name}').json())["count"]

    # Obtém as urls de todos os recursos disponíveis no Endpoint
    response = requests.get(f'https://pokeapi.co/api/v2/{endpoint_name}?limit={max_page}')
    data = response.json()

    responses_list = []

    # Faz a requisição na API dos dados de cada recurso por meio das urls
    # e compõe uma lista com as respostas de todas as requisições
    for resource in data['results']:
        response = requests.get(resource["url"]).json()
        response_str_json = json.dumps(response)
        responses_list.extend([response_str_json])

    # Gerando raw dataframe
    responses_list_json = sc.parallelize(responses_list)
    df_raw = spark.read.json(responses_list_json, multiLine = True)
    return df_raw

Os Endpoints necessários para responder às perguntas são:
- [Pokémon](https://pokeapi.co/docs/v2#pokemon)
- [Pokémon-Species](https://pokeapi.co/docs/v2#pokemon-species)
- [Evolution-Chain](https://pokeapi.co/docs/v2#evolution-chains)
- [Types](https://pokeapi.co/docs/v2#types)
- [Version-Groups](https://pokeapi.co/docs/v2#version-groups)

In [0]:
# Executando função criada para cada Endpoint que vamos precisar
df_pokemon_raw = generate_dataframe_from_request("pokemon")
df_pokemon_species_raw = generate_dataframe_from_request("pokemon-species")
df_evol_chains_raw = generate_dataframe_from_request("evolution-chain")
df_types_raw = generate_dataframe_from_request("type")
df_version_groups_raw = generate_dataframe_from_request("version-group")

## Modelagem dos dados

### Endpoint "Pokémon"

#### Pokémon
- Dataframe que descreve as principais características de um Pokémon.
- Possui também informações de `species_id` e `evol_chain_id`, que relacionam esse dataframe com o `df_evol_chains_species`.
- *Cada "Pokémon" possui apenas um `species_id` e um `evol_chain_id`.*

OBS: Foi observado que, para todos os Pokémon em `df_pokemon` que estão em sua forma padrão, ou seja,
`is_default = True`, temos que `pokemon_id = species_id`.

In [0]:
df_pokemon = df_pokemon_raw.selectExpr('id as pokemon_id',
                                       'name as pokemon_name',
                                       'is_default',
                                       'height',
                                       'base_experience',
                                       "regexp_extract(species.url, r'\/(\d+)', 1) as species_id")\

df_species = df_pokemon_species_raw.selectExpr('id as species_id',
                                               'name as species_name',
                                               "cast(regexp_extract(evolution_chain.url, r'\/(\d+)', 1) as long) as evol_chain_id")

df_pokemon = df_pokemon.join(df_species, ['species_id'], 'left')\
                       .select("pokemon_id",
                               "pokemon_name",
                               "is_default",
                               "height",
                               "base_experience",
                               "evol_chain_id",
                               "species_id",
                               "species_name")

df_pokemon.limit(20).display()

pokemon_id,pokemon_name,is_default,height,base_experience,evol_chain_id,species_id,species_name
19,rattata,true,3,51,7,19,rattata
7,squirtle,true,5,63,3,7,squirtle
6,charizard,true,17,267,2,6,charizard
9,blastoise,true,16,265,3,9,blastoise
17,pidgeotto,true,11,122,6,17,pidgeotto
5,charmeleon,true,11,142,2,5,charmeleon
1,bulbasaur,true,7,64,1,1,bulbasaur
10,caterpie,true,3,39,4,10,caterpie
3,venusaur,true,20,263,1,3,venusaur
12,butterfree,true,11,198,4,12,butterfree


#### Pokémon - Stats
- Possui as "stats" e seus respectivos valores base, para cada Pokémon.
- Por meio da coluna `pokemon_id`, relaciona-se com o `df_pokemon`.
- *Cada "Pokémon" possui um ou mais "stats" e cada "stat" está relacionada a um ou mais "Pokémon".*

<img src="https://i.postimg.cc/GpScVMFy/pokemon-stat.png" width = "600">

In [0]:
df_pokemon_stats = df_pokemon_raw.selectExpr('id as pokemon_id',
                                             'stats')\
                                 .select('*',
                                         explode_outer('stats').alias('stats_exploded'))\
                                 .selectExpr('*',
                                             "cast(regexp_extract(stats_exploded.stat.url, r'\/(\d+)', 1) as long) as stat_id",
                                             'stats_exploded.base_stat as stat_base',
                                             'stats_exploded.stat.name as stat_name')\
                                 .distinct()\
                                 .select('pokemon_id',
                                         'stat_id',
                                         'stat_name',
                                         'stat_base')
df_pokemon_stats.limit(20).display()

pokemon_id,stat_id,stat_name,stat_base
4,1,hp,39
1,2,attack,49
2,3,defense,63
3,2,attack,82
1,6,speed,45
3,5,special-defense,100
1,1,hp,45
2,4,special-attack,80
4,6,speed,65
1,4,special-attack,65


#### Pokémon - Moves

- Possui os golpes ("moves") para cada Pokémon.
- Relaciona-se com o `df_pokemon` por meio da coluna `pokemon_id`.
- *Cada "Pokémon" possui um ou mais "moves" e cada `move_id` está associado a um ou mais "Pokémon"*

<img src="https://i.postimg.cc/QMKPN0L5/pokemon-move.png" width = "600">

In [0]:
df_pokemon_moves = df_pokemon_raw.selectExpr('id as pokemon_id',
                                              'moves')\
                                 .select('pokemon_id',
                                         explode_outer('moves').alias('moves'))\
                                 .selectExpr('pokemon_id',
                                             "cast(regexp_extract(moves.move.url, r'\/(\d+)', 1) as long) as move_id",
                                             'moves.move.name as move_name')\
                                 .select('pokemon_id',
                                         'move_id',
                                         'move_name')\
                                 .selectExpr('pokemon_id',
                                             'move_id',
                                             'move_name')\
                                 .distinct()

df_pokemon_moves.limit(100).display()

pokemon_id,move_id,move_name
1,124,sludge
2,412,energy-ball
1,241,sunny-day
1,13,razor-wind
1,580,grassy-terrain
1,80,petal-dance
1,200,outrage
1,311,weather-ball
2,216,return
1,164,substitute


#### Moves - Version Groups

- Possui as características de cada move: `move_learn_method` e `version_group_id`.
- Relaciona-se com o `df_pokemon_moves` por meio da coluna `move_id` e com o `df_versions` por meio da `version_group_id`.
- *Cada "move" está relacionado a um ou mais "version_groups" e cada "version_group" está relacionado a um ou mais "moves"*.

<img src="https://i.postimg.cc/HWQKsz5x/version-group-move.png" width = "600">

In [0]:
df_moves_version_groups = df_pokemon_raw.selectExpr('moves')\
                                        .select(explode_outer('moves').alias('moves'))\
                                        .selectExpr("cast(regexp_extract(moves.move.url, r'\/(\d+)', 1) as long) as move_id",
                                                    'moves.version_group_details as version_group_details')\
                                        .select('move_id',
                                                explode_outer('version_group_details').alias('version_group_details_exploded'))\
                                        .selectExpr('move_id',
                                                    'version_group_details_exploded.move_learn_method.name as move_learn_method',
                                                    "cast(regexp_extract(version_group_details_exploded.version_group.url, r'\/(\d+)', 1) as long) as version_group_id")\
                                        .distinct()

df_moves_version_groups.limit(20).display()

move_id,move_learn_method,version_group_id
267,egg,20
282,tutor,16
73,level-up,19
74,level-up,11
76,level-up,6
237,machine,16
320,egg,11
345,egg,5
77,level-up,12
210,machine,3


### Endpoint "Version groups"

#### Versions
- Possui as "versions" que compõem cada "version_group".
- Relaciona-se com a `df_moves_version_groups` por meio da coluna `version_group_id`.
- *Uma "version" está associada a apenas um "version_group", e um "version_group" possui uma ou mais "versions".*

<img src="https://i.postimg.cc/KvxWKTr5/version-group-version.png" width = "600">

In [0]:
df_versions = df_version_groups_raw.select(col('id').alias('version_group_id'),
                                           col('name').alias('version_group_name'),
                                           explode_outer('versions').alias('versions_exploded'))\
                                   .selectExpr("cast(regexp_extract(versions_exploded.url, r'\/(\d+)', 1) as long) as version_id",
                                               'versions_exploded.name as version_name',
                                               'version_group_id',
                                               'version_group_name')

df_versions.limit(10).display()

version_id,version_name,version_group_id,version_group_name
1,red,1,red-blue
2,blue,1,red-blue
3,yellow,2,yellow
4,gold,3,gold-silver
5,silver,3,gold-silver
6,crystal,4,crystal
7,ruby,5,ruby-sapphire
8,sapphire,5,ruby-sapphire
9,emerald,6,emerald
10,firered,7,firered-leafgreen


#### Pokémon - Types

- Possui os "types" de cada "Pokémon".
- Relaciona-se com o `df_pokemon` por meio da coluna `pokemon_id`.
- Relaciona-se com o `df_types_damaged_types` por meio do `type_id`.
- *Um "Pokémon" pode ter um ou mais "types" e um "type" está associado a um ou mais "Pokémon"*.

<img src="https://i.postimg.cc/vTLSq8M9/pokemon-type.png" width = "600">

In [0]:
df_pokemon_types = df_pokemon_raw.selectExpr('id as pokemon_id',
                                             'types')\
                                 .select('*',
                                         explode_outer('types').alias('types_exploded'))\
                                 .selectExpr('*',
                                             'types_exploded.type.name as type_name',
                                             "cast(regexp_extract(types_exploded.type.url, r'\/(\d+)', 1) as long) as type_id")\
                                 .distinct()\
                                 .select('pokemon_id',
                                         'type_id',
                                         'type_name')

df_pokemon_types.limit(20).display()

pokemon_id,type_id,type_name
12,7,bug
2,4,poison
11,7,bug
1,4,poison
6,10,fire
3,12,grass
5,10,fire
3,4,poison
6,3,flying
8,11,water


### Endpoint "Types"

#### Types - Damaged Types
- Dataframe que lista, para cada tipo de "Pokémon", os "types" contra os quais ele possui vantagem ("damaged_types").
- Relaciona-se com a `df_pokemon_types` por meio da coluna `type_id`.
- *Um "type" pode ter vantagem contra nenhum ou vários "damaged_types" e um "damaged_type" está associado a um ou mais "types".*

<img src="https://i.postimg.cc/GhK5vpXn/type-type.png" width = "380">

In [0]:
df_types_damaged_types = df_types_raw.selectExpr('id as type_id',
                                              'damage_relations.double_damage_to')\
                                  .select('*',
                                          explode_outer('double_damage_to').alias('double_damage_exploded'))\
                                  .selectExpr('type_id',
                                              "cast(regexp_extract(double_damage_exploded.url, r'\/(\d+)', 1) as long) as damaged_type_id",
                                              'double_damage_exploded.name as damaged_type_name')

df_types_damaged_types.limit(20).display()

type_id,damaged_type_id,damaged_type_name
1,null,null
2,1,normal
2,6,rock
2,9,steel
2,15,ice
2,17,dark
3,2,fighting
3,7,bug
3,12,grass
4,12,grass


### Endpoint "Evolution Chains"

#### Evolution Chains - Species

- Descreve os possíveis caminhos evolutivos de cada "Pokémon Species", sendo cada caminho evolutivo formado por:
    - `evol_chain_id`: o identificador da cadeia evolutiva de cada Pokémon
    - `species_1`: espécie que o Pokémon assume na primeira forma (uma e apenas uma)
    - `species_2`: espécie que o Pokémon assume na segunda forma (0 ou várias)
    - `species_3`: espécie que o Pokémon assume na terceira forma (0 ou várias)
- Relaciona-se com o `df_pokemon` por meio da coluna `evol_chain_id` e pelas colunas de `species_id`.
- *Um Pokémon possui um e apenas um `evol_chain_id`, mas pode ter um ou mais caminhos evolutivos, logo, na tabela `df_evol_chains_species`, podemos ter mais de uma linha com o mesmo `evol_chain_id`, com caminhos diferentes para a evolução das "species".*
- *E uma "evolution_chain" está associada a pelo menos um "Pokémon".*

<img src="https://i.postimg.cc/0NF3rHc6/pokemon-evolution-chain.png" width = "600">



<img src="https://i.postimg.cc/Z5qDHhQB/pokemon-species.png" width = "600">

In [0]:
df_evol_chains_species = df_evol_chains_raw.selectExpr('id as evol_chain_id',
                                                       "chain.species as species_1",
                                                       "chain.evolves_to as species_2")\
                                           .select("evol_chain_id",
                                                   "species_1",
                                                   explode_outer(col("species_2")).alias("species_2"))\
                                           .selectExpr("evol_chain_id",
                                                       "species_1",
                                                       "species_2.species as species_2",
                                                       "species_2.evolves_to as species_3")\
                                           .select("evol_chain_id",
                                                    "species_1",
                                                    "species_2",
                                                    explode_outer(col("species_3")).alias("species_3"))\
                                           .selectExpr("evol_chain_id",
                                                       "species_1",
                                                       "species_2",
                                                       "species_3.species as species_3")\
                                           .selectExpr("evol_chain_id",
                                                      "cast(regexp_extract(species_1.url, r'\/(\d+)', 1) as long) as species_1_id",
                                                      "species_1.name as species_1_name",
                                                      "cast(regexp_extract(species_2.url, r'\/(\d+)', 1) as long) as species_2_id",
                                                      "species_2.name as species_2_name",
                                                      "cast(regexp_extract(species_3.url, r'\/(\d+)', 1) as long) as species_3_id",
                                                      "species_3.name as species_3_name")

df_evol_chains_species.limit(20).display()

evol_chain_id,species_1_id,species_1_name,species_2_id,species_2_name,species_3_id,species_3_name
1,1,bulbasaur,2,ivysaur,3,venusaur
2,4,charmander,5,charmeleon,6,charizard
3,7,squirtle,8,wartortle,9,blastoise
4,10,caterpie,11,metapod,12,butterfree
5,13,weedle,14,kakuna,15,beedrill
6,16,pidgey,17,pidgeotto,18,pidgeot
7,19,rattata,20,raticate,null,null
8,21,spearow,22,fearow,null,null
9,23,ekans,24,arbok,null,null
10,172,pichu,25,pikachu,26,raichu


<img src="https://i.postimg.cc/RZ7RFK3q/ER-diagram-pokemon.png"/>

## Perguntas

### Pergunta 1
- Quais são os maiores Pokémon? Exiba os Pokémon e a altura.

In [0]:
all = Window.partitionBy(lit(1))

df_1 = df_pokemon.select('pokemon_name',
                         'height',
                         _max('height').over(all).alias("max_height"))\
                 .where('height = max_height')\
                 .drop('max_height')

df_1.display()

pokemon_name,height
eternatus-eternamax,1000


### Pergunta 2

- Quantos Pokémon possuem mais de um caminho evolutivo, ou seja, mais de uma
segunda ou terceira forma? Informe a quantidade, tomando como base o número
distinto de Pokémon na sua primeira forma.

In [0]:
window_evol_chain = Window.partitionBy(df_evol_chains_species.evol_chain_id)

df_2 = df_evol_chains_species.select('evol_chain_id',
                                     count("*").over(window_evol_chain).alias("evol_chains_qty"),
                                     'species_1_id')\
                             .distinct()\
                             .join(df_pokemon, df_pokemon.species_id == df_evol_chains_species.species_1_id, 'left')\
                             .where("evol_chains_qty > 1 AND is_default = TRUE")\
                             .select("pokemon_name",
                                     "evol_chains_qty")
df_2.display()
answer = df_2.count()

print(f"Resposta: O número de Pokémon com mais de um caminho evolutivo é: {answer}.")

pokemon_name,evol_chains_qty
oddish,2
meowth,2
slowpoke,2
eevee,8
poliwag,2
tyrogue,3
ralts,2
wurmple,2
burmy,2
snorunt,2


Resposta: O número de Pokémon com mais de um caminho evolutivo é: 15.


### Pergunta 3

- Quais são os Pokémon do tipo gelo (ice) que mais fornecem experiência ao serem
derrotados? Leve em consideração apenas os Pokémon que estão em sua forma
padrão. Exiba os Pokémon e a experiência que é fornecida.

In [0]:
df_3 = df_pokemon_types.where("type_name = 'ice'")\
                       .join(df_pokemon, ['pokemon_id'], 'left')\
                       .where("is_default = TRUE")\
                       .select("pokemon_name",
                               "base_experience",
                               _max('base_experience').over(all).alias("max_base_exp"))\
                       .where('base_experience = max_base_exp')\
                       .drop('max_base_exp')

df_3.display()

pokemon_name,base_experience
kyurem,330


### Pergunta 4

- Encontre o golpe (move) que é mais aprendido por Pokémon pelo método level-up e em quais versões do jogo
isso acontece, levando em consideração apenas os Pokémon que estão em sua forma padrão. Em seguida, entre os
Pokémon que aprendem esse golpe (por level-up e nas versões do jogo encontrada no passo anterior), diga quais
são os Pokémon que possuem o maior valor do atributo (stat) ‘attack’. Exiba os Pokémon, as versões do jogo
em que acontece de mais Pokémon aprenderem o golpe e qual é o valor do atributo ‘attack’ deles.

In [0]:
window_move_version_group = Window.partitionBy(df_pokemon_moves.move_id, df_moves_version_groups.version_group_id)

df_4 =  df_pokemon_moves.join(df_pokemon, ['pokemon_id'], 'left')\
                        .where("is_default = TRUE")\
                        .join(df_moves_version_groups, ['move_id'], 'left').distinct()\
                        .where("move_learn_method = 'level-up'")\
                        .select('pokemon_id',
                                'pokemon_name',
                                'move_name',
                                'move_learn_method',
                                'version_group_id',
                                count("pokemon_id").over(window_move_version_group).alias('pokemon_qty'))\
                        .select('*',
                                _max('pokemon_qty').over(all).alias("max_pokemon_qty"))\
                        .where("pokemon_qty = max_pokemon_qty")\
                        .join(df_pokemon_stats, ['pokemon_id'], 'left')\
                        .where("stat_name = 'attack'")\
                        .select('*',
                                _max('stat_base').over(all).alias("max_stat_base"))\
                        .where("stat_base = max_stat_base")\
                        .join(df_versions, ['version_group_id'], 'left')\
                        .select('pokemon_name',
                                'version_name',
                                'stat_base')

df_4.display()

pokemon_name,version_name,stat_base
kartana,emerald,181
kartana,pearl,181
kartana,diamond,181
kartana,colosseum,181
kartana,white-2,181
kartana,black-2,181
kartana,sapphire,181
kartana,ruby,181
kartana,leafgreen,181
kartana,firered,181


### Pergunta 5

- Quais são as 7 evoluções que geram o maior aumento em algum dos atributos? Por
exemplo, um Pokémon com o stat ‘attack’ igual a 50 evoluir para um com o stat ‘attack’
igual a 150, o aumento seria de 100. A análise deve ser feita para todos os atributos de
todos os Pokémon, e somente aqueles na sua forma padrão. Exiba a pré-evolução, a
evolução, o atributo afetado e o quanto ele aumentou com a evolução.

In [0]:
# Considerando a observação de que para todos os Pokémons em df_pokemon
# que estão em sua forma padrão, ou seja, is_default = True, temos que
# pokemon_id = species_id. Dessa forma, podemos considerar que tanto o 
# nome quanto o id do Pokémon são iguais ao nome e ao id da sua espécie.

# Obtendo dataframe com todas as evoluções possíveis: 
# primeira forma -> segunda forma ou segunda forma -> terceira forma.
df_5 = df_evol_chains_species.selectExpr("species_1_id as pre_evolution_id",
                                         "species_1_name as pre_evolution_pokemon",
                                         "species_2_id as evolution_id",
                                         "species_2_name as evolution_pokemon")

df_5_1 = df_evol_chains_species.selectExpr("species_2_id as pre_evolution_id",
                                           "species_2_name as pre_evolution_pokemon",
                                           "species_3_id as evolution_id",
                                           "species_3_name as evolution_pokemon")
df_5 = df_5.union(df_5_1)\
           .distinct()\
           .where(col("pre_evolution_id").isNotNull()&\
                  col("evolution_id").isNotNull())\
           .join(df_pokemon_stats, df_pokemon_stats.pokemon_id == df_5.pre_evolution_id, 'left')\
           .selectExpr("pre_evolution_pokemon",
                       "stat_name as pre_stat_name",
                       "stat_base as pre_stat_base",
                       "evolution_id",
                       "evolution_pokemon")\
           .join(df_pokemon_stats, df_pokemon_stats.pokemon_id == df_5.evolution_id, 'left')\
           .selectExpr("pre_evolution_pokemon",
                       "pre_stat_name",
                       "pre_stat_base",
                       "evolution_pokemon",
                       "stat_name as stat_name_after",
                       "stat_base as stat_base_after")\
           .where("pre_stat_name = stat_name_after")\
           .selectExpr("pre_evolution_pokemon",
                       "evolution_pokemon",
                       "pre_stat_name as stat",
                       "(stat_base_after - pre_stat_base) as stat_increase")\
           .orderBy("stat_increase", ascending=False)\
           .limit(7)

df_5.display()

pre_evolution_pokemon,evolution_pokemon,stat,stat_increase
happiny,chansey,hp,150
nincada,ninjask,speed,120
shelmet,accelgor,speed,120
magikarp,gyarados,attack,115
cosmoem,lunala,special-attack,108
cosmoem,solgaleo,attack,108
cosmog,cosmoem,special-defense,100


### Pergunta 6

- Quais são os Pokémon com maior vantagem atacante sobre outros Pokémon? Leve em
consideração apenas o tipo do Pokémon (Um Pokémon do tipo fogo/lutador
(fire/fighting) possui vantagem atacante em um Pokémon inseto/voador (bug/flying),
pois o tipo fogo tem vantagem sobre inseto, apesar do tipo lutador não ser eficiente
contra o tipo inseto e voador) e apenas os Pokémon que estão em sua forma padrão.
Exiba os Pokémon, e a quantidade de Pokémon que eles possuem vantagem atacante
sobre.

In [0]:
df_6 = df_pokemon_types.join(df_pokemon, ['pokemon_id'], 'left')\
                       .where("is_default = TRUE")\
                       .join(df_types_damaged_types, ['type_id'], 'left')\
                       .selectExpr("pokemon_id as pokemon_id_1",
                                   "pokemon_name as pokemon_name_1",
                                   "type_name as type_pokemon",
                                   "damaged_type_id")\
                       .join(df_pokemon_types, df_pokemon_types.type_id == df_types_damaged_types.damaged_type_id, 'left')\
                       .join(df_pokemon, ['pokemon_id'], 'left')\
                       .where("is_default = TRUE")\
                       .selectExpr("pokemon_name_1 as pokemon_name",
                                   "pokemon_id as pokemon_id_damaged")\
                       .distinct()\
                       .groupBy("pokemon_name")\
                       .agg(count("pokemon_id_damaged").alias("disadvantaged_pokemon_qty"))\
                       .withColumn("max_num_pokemon", _max('disadvantaged_pokemon_qty').over(all))\
                       .where('disadvantaged_pokemon_qty = max_num_pokemon')\
                       .drop('max_num_pokemon')

df_6.display()

pokemon_name,disadvantaged_pokemon_qty
crabominable,546
